In [1]:
import pandas as pd
from ipywidgets import Video, Image 
!jupyter nbextension list
# (you need to see both jupyter-js-widgets/extension and jupyter-video/extension enabled and validated below)
# !jupyter nbextension enable --py --sys-prefix widgetsnbextension
# !pip install Jupyter-Video_Widget
# !jupyter nbextension enable --py --sys-prefix jpy_video

Known nbextensions:
  config dir: /root/anaconda3/etc/jupyter/nbconfig
    notebook section
      jupyter-js-widgets/extension  enabled 
      - Validating: OK
      jupyter-video/extension  enabled 
      - Validating: OK


In [21]:
gt = pd.read_csv("ground_truth.csv",header=0,index_col=1)

In [22]:
gt.head()

,video-UID,labeler,class
clip_title,,,
5yo_vs_Mickey_slice0_slice0.mp4,10000,Alex,0
5yo_vs_Mickey_slice10_slice0.mp4,10001,Alex,0
5yo_vs_Mickey_slice15_slice0.mp4,10002,Alex,0
5yo_vs_Mickey_slice20.mp4,10003,Ahsen,0
5yo_vs_Mickey_slice5.mp4,10004,Jen,0


In [23]:
gt['class'].value_counts()

 0    3783
 1    1049
 3     266
 4     147
-1     144
 2      52
Name: class, dtype: int64

In [24]:
df = pd.read_csv("eval/punch101_test_preds.txt",header=0)
df['filename'] = df['Path'].apply(lambda x: x.split("/")[1].replace("_scaled",".mp4"))
df.drop('Path',inplace=True,axis=1)
df.set_index('filename',inplace=True)

In [25]:
df.head()

,RGB_Preds,Flow_Preds,Combined_Preds,Truth
filename,,,,
heavy_bag_drills_slice105.mp4,1,1,1,1
NV_630.mp4,0,0,0,0
V_980.mp4,3,0,0,0
streetfight5_slice680.mp4,0,0,0,0
SouthAmericanDrumline_slice35.mp4,0,0,0,0


In [26]:
df.shape

(1060, 4)

In [27]:
df = df.join(gt).reset_index().set_index('video-UID')
df.rename(columns={'filename':'clip_title'},inplace=True)

In [28]:
df.shape

(1060, 7)

## Jen was concerned that the "Truth" column coming from Lance's 'punch101_test_preds.txt' file did not always match the "class" column coming from our 'ground_truth.csv' master labeled datasest.

In [30]:
df[df['Truth'] != df['class']].head(10)

,clip_title,RGB_Preds,Flow_Preds,Combined_Preds,Truth,labeler,class
video-UID,,,,,,,
13432,heavy_bag_drills_slice105.mp4,1,1,1,1,Lance,3
12194,V_619_slice0.mp4,3,0,0,3,Lance,1
12596,V_919.mp4,0,3,3,3,Alex,1
12381,V_789_slice100.mp4,3,3,3,3,Alex,1
14174,perfect_hook_punch_slice260.mp4,0,0,0,2,Jen,4
15159,streetfight5_slice55.mp4,3,0,3,3,Lance,1
12221,V_642.mp4,3,0,0,3,Alex,1
15180,streetfight5_slice65.mp4,3,0,3,3,Jen,1
12902,downward_strike_punches_slice250.mp4,1,1,1,1,Jen,3


In [11]:
# df[df['Truth'] != df['class']].shape

(297, 7)

In [12]:
# Play a few examples from above "Truth"/"class" mismatch subset
# Jen is pretty sure "class" is what we should be measuring against
# clip = 'streetfight3_slice525.mp4'
# clip = 'V_345.mp4'
# clip = 'fast_punch_drill_slice160.mp4'

In [13]:
# show = Video.from_file('/root/vids/' + clip)
# show

Video(value=b'\x00\x00\x00 ftypisom\x00\x00\x02\x00isomiso2avc1mp41\x00\x00\x00\x08free\x00\x0f.8mdat\x00\x00\…

## Turns out that we had a different mapping in Lance's file 
* 0 NoPunch
* 3 ViolentPunchContact
* 4 ViolentPunchNoContact
* 1 NonViolentPunchContact
* 2 NonViolentPunchNoContact

## So remapping those columns below

In [31]:
df.columns

Index(['clip_title', 'RGB_Preds', 'Flow_Preds', 'Combined_Preds', 'Truth',
       'labeler', 'class'],
      dtype='object')

In [32]:
def translate_to_team_labels(n):
    if n == 0:
        return 0
    elif n == 1:
        return 3
    elif n == 2:
        return 4
    elif n == 3:
        return 1
    elif n == 4:
        return 2
    

df['nRGB_Preds'] = df['RGB_Preds'].apply(lambda x: translate_to_team_labels(x))
df['nFlow_Preds'] = df['Flow_Preds'].apply(lambda x: translate_to_team_labels(x))
df['nCombined_Preds'] = df['Combined_Preds'].apply(lambda x: translate_to_team_labels(x))
df['nTruth'] = df['Truth'].apply(lambda x: translate_to_team_labels(x))

In [34]:
df[df['class'] != df['nTruth']].shape

(0, 11)

In [35]:
df.drop(columns = ['RGB_Preds','Flow_Preds','Combined_Preds','Truth'],inplace = True)

In [40]:
df.rename(columns = {'nRGB_Preds':'RGB_Preds','nFlow_Preds':'Flow_Preds', \
                    'nCombined_Preds':'Combined_Preds','nTruth':'Truth'},inplace=True)

In [41]:
df.head()

,clip_title,labeler,class,RGB_Preds,Flow_Preds,Combined_Preds,Truth
video-UID,,,,,,,
13432,heavy_bag_drills_slice105.mp4,Lance,3,3,3,3,3
10918,NV_630.mp4,Jen,0,0,0,0,0
12665,V_980.mp4,Alex,0,1,0,0,0
15187,streetfight5_slice680.mp4,Ahsen,0,0,0,0,0
11420,SouthAmericanDrumline_slice35.mp4,Ahsen,0,0,0,0,0


In [42]:
df[df['class']!=df['Truth']]

,clip_title,labeler,class,RGB_Preds,Flow_Preds,Combined_Preds,Truth
video-UID,,,,,,,


In [44]:
df.drop(columns='Truth',inplace=True)

In [45]:
df.rename(columns={'Combined_Preds':'predix'},inplace = True)

In [46]:
df.head()

,clip_title,labeler,class,RGB_Preds,Flow_Preds,predix
video-UID,,,,,,
13432,heavy_bag_drills_slice105.mp4,Lance,3,3,3,3
10918,NV_630.mp4,Jen,0,0,0,0
12665,V_980.mp4,Alex,0,1,0,0
15187,streetfight5_slice680.mp4,Ahsen,0,0,0,0
11420,SouthAmericanDrumline_slice35.mp4,Ahsen,0,0,0,0


In [47]:
df.sample(10)

,clip_title,labeler,class,RGB_Preds,Flow_Preds,predix
video-UID,,,,,,
15039,streetfight4_slice55.mp4,Lance,1,0,0,0
11259,NV_935.mp4,Lance,0,0,0,0
14460,street_confrontation_punch_slice55.mp4,Ahsen,1,0,0,0
549,NV_32.mp4,Jen,0,0,0,0
15118,streetfight5_slice365.mp4,Ahsen,0,0,0,0
12984,downward_strike_punches_slice620.mp4,Ahsen,0,0,0,0
13351,groomsmen_dance_slice165.mp4,Alex,0,0,0,0
14567,streetfight1_slice465.mp4,Jen,0,0,0,0
14242,short_strikes_slice100.mp4,Lance,3,3,3,3


In [48]:
df.shape

(1060, 6)

In [49]:
df.to_csv('TSN_predix_multiclass.csv')

In [50]:
df['Binary'] = df['predix'].apply(lambda x: int(x>0))

In [51]:
df.head()

,clip_title,labeler,class,RGB_Preds,Flow_Preds,predix,Binary
video-UID,,,,,,,
13432,heavy_bag_drills_slice105.mp4,Lance,3,3,3,3,1
10918,NV_630.mp4,Jen,0,0,0,0,0
12665,V_980.mp4,Alex,0,1,0,0,0
15187,streetfight5_slice680.mp4,Ahsen,0,0,0,0,0
11420,SouthAmericanDrumline_slice35.mp4,Ahsen,0,0,0,0,0


In [52]:
df['Binary_label'] = df['class'].apply(lambda x: int(x>0))

In [53]:
df.head()

,clip_title,labeler,class,RGB_Preds,Flow_Preds,predix,Binary,Binary_label
video-UID,,,,,,,,
13432,heavy_bag_drills_slice105.mp4,Lance,3,3,3,3,1,1
10918,NV_630.mp4,Jen,0,0,0,0,0,0
12665,V_980.mp4,Alex,0,1,0,0,0,0
15187,streetfight5_slice680.mp4,Ahsen,0,0,0,0,0,0
11420,SouthAmericanDrumline_slice35.mp4,Ahsen,0,0,0,0,0,0


In [54]:
df.sample(10)

,clip_title,labeler,class,RGB_Preds,Flow_Preds,predix,Binary,Binary_label
video-UID,,,,,,,,
15185,streetfight5_slice670.mp4,Lance,0,0,0,0,0,0
13881,kicks_line_dance_slice105.mp4,Lance,0,0,0,0,0,0
10036,Australian_flash_mob_dance_slice235.mp4,Alex,0,0,0,0,0,0
11174,NV_858.mp4,Jen,0,0,0,0,0,0
12466,V_80_slice0.mp4,Alex,1,1,0,1,1,1
15067,streetfight5_slice135.mp4,Jen,0,0,0,0,0,0
11127,NV_815.mp4,Ahsen,0,0,0,0,0,0
11782,V_248.mp4,Ahsen,1,1,1,1,1,1
4292,street_confrontation_punch_slice20.mp4,Ahsen,0,0,0,0,0,0


In [55]:
df.drop(columns = ['class','predix'],inplace = True)

In [56]:
df.rename(columns = {'Binary':'predix','Binary_label':'label'},inplace=True)

In [57]:
df.head()

,clip_title,labeler,RGB_Preds,Flow_Preds,predix,label
video-UID,,,,,,
13432,heavy_bag_drills_slice105.mp4,Lance,3,3,1,1
10918,NV_630.mp4,Jen,0,0,0,0
12665,V_980.mp4,Alex,1,0,0,0
15187,streetfight5_slice680.mp4,Ahsen,0,0,0,0
11420,SouthAmericanDrumline_slice35.mp4,Ahsen,0,0,0,0


In [58]:
df.to_csv('TSN_predix_binary.csv')

In [59]:
!ls

 AWOL_labeled_videos.csv
"Ahsen's MovieHouseMcLabelMore.ipynb"
"Alex's MovieHouseMcLabelMore.ipynb"
 CNN_singleImage_baseline.py
 CNN_single_baseline.ipynb
 ClassCharacterization.ipynb
 ClipAssigner.ipynb
 Countdown_to_truthiness.ipynb
 CustomAnnotationsAndFolderStructure-Alex.ipynb
"JenD's MovieHouseMcLabelMore.ipynb"
 Join_TSNpredix_and_GTdb.ipynb
 Joiner.ipynb
"Lance's MovieHouseMcLabelMore.ipynb"
 README.md
 TSN_predix_binary.csv
 TSN_predix_multiclass.csv
 ahsen_clips.csv
 ahsen_week11_labeling_assignment.csv
 ahsen_week9_clips.csv
 alex_clips.csv
 alex_week11_labeling_assignment.csv
 alex_week9_clips.csv
 all_clips.txt
 baseline_model_prediction.ipynb
 convertCodec.sh
 did_not_label_in_week6.csv
 eval
 first_1k_labeled.csv
 first_1k_labeled_long_vids_removed.csv
 full_labels.csv
 ground_truth.csv
 importFromIBM.ipynb
 importFromIBM.py
 jen_clips.csv
 jen_week9_clips.csv
 jend_week11_labeling_assignment.csv
 lambda_function.py
 lance_clips.csv
 lance_week11_labeling_assignment.csv